In [1]:
import cv2
import dlib
import time


In [26]:
# Descargar herramientas de c++https://visualstudio.microsoft.com/es/visual-cpp-build-tools/
# Descargar el modelo desde: http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
import cv2
import dlib
from deepface import DeepFace
import numpy as np
import random

def overlay_image(background, overlay, pos_x, pos_y):
    """
    Overlay an image with an alpha channel over another image at a specific position,
    ensuring it does not go out of bounds of the frame.
    """
    h, w = overlay.shape[:2]

    # Crop the overlay if it goes out of the frame bounds
    if pos_y < 0:
        overlay = overlay[-pos_y:, :]
        pos_y = 0
    if pos_x < 0:
        overlay = overlay[:, -pos_x:]
        pos_x = 0
    if pos_y + h > background.shape[0]:
        overlay = overlay[:background.shape[0] - pos_y, :]
    if pos_x + w > background.shape[1]:
        overlay = overlay[:, :background.shape[1] - pos_x]

    # Apply the overlay using the alpha channel
    alpha_overlay = overlay[:, :, 3] / 255.0  # Alpha channel for transparency
    for c in range(0, 3):
        background[pos_y:pos_y + overlay.shape[0], pos_x:pos_x + overlay.shape[1], c] = \
            (1 - alpha_overlay) * background[pos_y:pos_y + overlay.shape[0], pos_x:pos_x + overlay.shape[1], c] + \
            alpha_overlay * overlay[:, :, c]
    return background


def load_image(path):
    return cv2.imread(path, cv2.IMREAD_UNCHANGED)  # Complete path to Chopper's hat

def zoro_scar_filter(frame, landmarks):
    # Posiciones clave para la cicatriz y el ojo
    forehead = (landmarks.part(19).x, landmarks.part(19).y)
    cheek = (landmarks.part(31).x, landmarks.part(31).y)
    eye_top = landmarks.part(37).y  # Punto superior del ojo izquierdo
    eye_bottom = landmarks.part(41).y  # Punto inferior del ojo izquierdo

    scar_color = (50, 50, 52)
    scar_thickness = 2
    num_lines = 4
    
    # Segmento superor de la cicatriz (justo encima del ojo) 
    for i in range(num_lines):
        offset_x = random.randint(-1, 1)  # Variación horizontal para simular textura de cicatriz
        start_point = (forehead[0] + offset_x, forehead[1] + i * 5)
        end_point = (forehead[0] + offset_x, eye_top - 5 - i * 2)

        cv2.line(frame, start_point, end_point, scar_color, scar_thickness)

    # Segmento inferior de la cicatriz (justo bajo el ojo)
    for i in range(num_lines):
        offset_x = random.randint(-1, 1)
        start_point = (forehead[0] + offset_x, eye_bottom + 5 + i * 3)
        end_point = (forehead[0] + offset_x, cheek[1] - i * 2)

        cv2.line(frame, start_point, end_point, scar_color, scar_thickness)
    

def choper_hat_filter(frame, landmarks):
    chopper_hat = load_image("chopper_hat.png")

    left_eye = (landmarks.part(36).x, landmarks.part(36).y)
    left_temple = (landmarks.part(17).x, landmarks.part(17).y)  # Left temple point
    right_temple = (landmarks.part(26).x, landmarks.part(26).y)  # Right temple point
    chin = (landmarks.part(8).x, landmarks.part(8).y)

    head_width = int(2.5 * (right_temple[0] - left_temple[0]))  # Increased width for better coverage
    head_height = int(1.5 * abs(chin[1] - left_eye[1]))   # Adjusted height for appropriate look
    chopper_hat = cv2.resize(chopper_hat, (head_width, head_height))
    
    # Position the hat: centered over the forehead and head
    pos_x = left_temple[0] - (head_width *2 // 7)  # Center horizontally
    pos_y = left_eye[1] - int(head_height * 0.95)  # Position higher to cover the top of the head

    # Overlay the hat on the image
    overlay_image(frame, chopper_hat, pos_x, pos_y)


def afro_brook_filter(frame, landmarks):
    afro_brook = load_image("afro_brook.png")
    
    left_eye = (landmarks.part(36).x, landmarks.part(36).y)
    left_temple = (landmarks.part(17).x, landmarks.part(17).y)  
    right_temple = (landmarks.part(26).x, landmarks.part(26).y) 
    chin = (landmarks.part(8).x, landmarks.part(8).y)

    head_width = int(2.5 * (right_temple[0] - left_temple[0])) 
    head_height = int(1.5 * abs(chin[1] - left_eye[1]))  
    afro_brook = cv2.resize(afro_brook, (int(head_width*1.4), int(head_height*1.8)))
    

    pos_x = left_temple[0] - (head_width *2 // 4)  
    pos_y = left_eye[1] - int(head_height * 1.3)  

    overlay_image(frame, afro_brook, pos_x, pos_y)


def luffy_hat_filter(frame, landmarks):
    pass

def luffy_scar_filter(frame, landmarks):
    pass

function_list = [ afro_brook_filter, zoro_scar_filter, choper_hat_filter, luffy_hat_filter, luffy_scar_filter]

predictor = dlib.shape_predictor( "shape_predictor_68_face_landmarks.dat")

video_capture = cv2.VideoCapture(0)
index = 0  # Índice inicial para function_list

while True:
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    try:
        # Extraer las caras usando DeepFace
        faces = DeepFace.extract_faces(frame, detector_backend='yolov8')
        
        for face in faces:
            x, y = face['facial_area']['x'], face['facial_area']['y']
            w, h = face['facial_area']['w'], face['facial_area']['h']
            dlib_rect = dlib.rectangle(left=x, top=y, right=x + w, bottom=y + h)
            
            landmarks = predictor(gray, dlib_rect)
            
            # Ejecutar la función actual en function_list basada en 'index'
            function_list[index](frame, landmarks)

    except Exception as e:
        print(e)

    # Mostrar la imagen con el filtro aplicado
    cv2.imshow("One Piece Filter", frame)
    
    # Leer la tecla presionada
    key = cv2.waitKey(5) & 0xFF

    if key == ord("q"):
        # Decrementar el índice y asegurarse de que esté en rango
        index = (index - 1) % len(function_list)
    elif key == ord("e"):
        # Incrementar el índice y asegurarse de que esté en rango
        index = (index + 1) % len(function_list)
    elif key == 27:  # Código ASCII para 'Esc'
        # Salir del bucle y cerrar la ventana
        break

video_capture.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import torch
from deepface import DeepFace
from ultralytics import YOLO

# Set device to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load YOLOv8 model for face detection
yolo_model = YOLO("yolov8n.pt")  # Adjust model as needed

# Load images for each filter item
luffy_hat = cv2.imread("luffy_hat.png", cv2.IMREAD_UNCHANGED)
luffy_scar = cv2.imread("luffy_scar.png", cv2.IMREAD_UNCHANGED)
zoro_scar = cv2.imread("zoro_scar.png", cv2.IMREAD_UNCHANGED)
sanji_eyebrow= cv2.imread("sanji_eyebrow.png", cv2.IMREAD_UNCHANGED)
afro_brook = cv2.imread("chopper_hat.png", cv2.IMREAD_UNCHANGED)

# Function to overlay image with rotation and scaling
def overlay_rotated_image(background, overlay, pos, angle=0, scale=1.0):
    h, w = overlay.shape[:2]
    overlay = cv2.resize(overlay, (int(w * scale), int(h * scale)))
    center = (overlay.shape[1] // 2, overlay.shape[0] // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_overlay = cv2.warpAffine(overlay, rotation_matrix, (overlay.shape[1], overlay.shape[0]), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)
    overlay_x, overlay_y = pos

    for y in range(rotated_overlay.shape[0]):
        for x in range(rotated_overlay.shape[1]):
            if overlay_y + y >= background.shape[0] or overlay_x + x >= background.shape[1]:
                continue
            alpha = rotated_overlay[y, x, 3] / 255.0
            background[overlay_y + y, overlay_x + x] = (1 - alpha) * background[overlay_y + y, overlay_x + x] + alpha * rotated_overlay[y, x, :3]
    return background

# Main loop
video_capture = cv2.VideoCapture(0)
while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    # Detect faces using YOLOv8
    results = yolo_model(frame)
    faces = []
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            faces.append((x1, y1, x2, y2))

    # Process each detected face
    for facial_area in faces:
        # Extract landmarks with DeepFace’s RetinaFace backend
        landmarks = None
        try:
            analysis = DeepFace.analyze(frame, actions=['emotion'], detector_backend="retinaface")
            landmarks = analysis[0]["region"]
        except Exception as e:
            print(f"Error detecting landmarks: {e}")

        if landmarks:
            # Position and scale Luffy's Hat
            left_temple = (landmarks["left_eye_x"], landmarks["left_eye_y"] - 20)
            right_temple = (landmarks["right_eye_x"], landmarks["right_eye_y"] - 20)
            hat_x = min(left_temple[0], right_temple[0]) - int(luffy_hat.shape[1] * 0.7)
            hat_y = left_temple[1] - luffy_hat.shape[0] // 2
            overlay_rotated_image(frame, luffy_hat, (hat_x, hat_y), scale=0.6)

            # Position and scale Zoro's Eye Scar
            scar_x = landmarks["left_eye_x"] - int(zoro_scar.shape[1] * 0.5)
            scar_y = landmarks["left_eye_y"]
            overlay_rotated_image(frame, zoro_scar, (scar_x, scar_y), scale=0.5)

            # Position and scale Sanji’s Curly Eyebrows
            eyebrow_scale = 0.6
            eyebrow_x_left = landmarks["left_eye_x"] - int(curly_eyebrow.shape[1] * eyebrow_scale // 2)
            eyebrow_y_left = landmarks["left_eye_y"] - 15
            eyebrow_x_right = landmarks["right_eye_x"] - int(curly_eyebrow.shape[1] * eyebrow_scale // 2)
            eyebrow_y_right = landmarks["right_eye_y"] - 15
            overlay_rotated_image(frame, curly_eyebrow, (eyebrow_x_left, eyebrow_y_left), scale=eyebrow_scale)
            overlay_rotated_image(frame, curly_eyebrow, (eyebrow_x_right, eyebrow_y_right), scale=eyebrow_scale)


            # Position and scale Chopper's Hat and Antlers
            hat_x_chopper = min(left_temple[0], right_temple[0]) - int(afro_brook.shape[1] * 0.7)
            hat_y_chopper = left_temple[1] - afro_brook.shape[0] // 2
            overlay_rotated_image(frame, afro_brook, (hat_x_chopper, hat_y_chopper), scale=0.6)

    # Display the frame with filters
    cv2.imshow("One Piece Filters", frame)
    if cv2.waitKey(5) & 0xFF == ord("q"):
        break

video_capture.release()
cv2.destroyAllWindows()


Using device: cpu

0: 480x640 1 person, 108.0ms
Speed: 3.0ms preprocess, 108.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)


KeyboardInterrupt: 